In [1]:
import seaborn as sns
import pandas as pd
import pylab as plt
import numpy as np
import steam.api
from steam.api import interface
import time
import os
import glob
import json
import tensorflow as tf
import sys
from IPython import display

#dont keep api key in github!!!
api_file = os.path.join('..','..','apikeys','steam_api_key.txt')
with open(api_file, 'r') as fin:
    steam.api.key.set(fin.readline())

/Users/bailey/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#declare interface urls
#dota2_beta should be used for testing in order to not aggressively make API requests
dota2_id = '570'
dota2_beta_id = '205790'

# Get 500 last matches to get approximate hero ratios
Need to be careful here so that test dataset doesn't have crossover with training data. My strategy to eliminate overlap was to start collecting match data for the training set for matches after the oldest match in the test set. To get around the 500 match retrieval limit I save 500 at a time to a .csv with a filename that has the start and end match_id. This means I can get 500, wait a bit, then get 500 more that are guarenteed to not have overlap. 

How long do I have to wait to get 500 more?? Unclear and was unable to find documentation. Thanks Valve.

In [3]:
#get matches data from dota2 API and put IDs into a dataframe
num_matches_to_request = 500

match_list = []

#put a start match_id if I'm trying to get more matches after getting 500
#if trying to load starting from a match_id previously obtained and saved
#use arg: start_at_match_id = old_match_list[-1]-1
matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(hero_id = 121,
                                                               game_mode = 1,
                                                               skill_level = 3,
                                                               min_players = 10,
                                                               matches_requested = 100)




#@TODO: check if lobby_type is public/ranked matching (0 or 7) before appending
[match_list.append(match['match_id']) for match in matches['result']['matches']]

current_num = 100

while current_num <= num_matches_to_request:
    
    #check that match request returned any matches before trying to get more
    if len(match_list)==0:
        print('No matches returned\nWait a bit and retry\n\n    GabeN Bless')
    
    #sleep so we don't overload the servers
    time.sleep(1)
                
    current_num += 100
    matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(start_at_match_id = match_list[-1]-1,
                                                                   game_mode = 1,
                                                                   skill_level = 3,
                                                                   min_players = 10,
                                                                   matches_requested = 100)

    #@TODO: check if lobby_type is public/ranked matching (0 or 7) before appending
    [match_list.append(match['match_id']) for match in matches['result']['matches']]

    
dota2_test_df = pd.DataFrame()
dota2_test_df['match_id'] = match_list

In [4]:
#get match details from match_ids obtained
#add radiant_win column
#add heroes split between dire and radiant

radiant_win_column = []
radiant_hero_column = []
dire_hero_column = []
match_details = []

for match_id in dota2_test_df['match_id']:
    time.sleep(1)
    match_details.append(interface('IDOTA2Match_' + dota2_id).GetMatchDetails(match_id = str(match_id)))
    
    radiant_win_column.append(match_details['result']['radiant_win'])
    
    radiant_hero_row = []
    dire_hero_row = []
    for player in match_details['result']['players']:
        #First bit of 8-bit integer player_slot is 1 if the player was on dire
        #so if player was on dire then player_slot > 128
        if player['player_slot']/128<1:
            radiant_hero_row.append(player['hero_id'])
        else:
            dire_hero_row.append(player['hero_id'])
    
    radiant_hero_column.append(radiant_hero_row)
    dire_hero_column.append(dire_hero_row)

TypeError: list indices must be integers or slices, not str

In [ ]:
#Add radiant_win column to Dataframe
dota2_test_df['radiant_win'] = 1*np.array(radiant_win_column)

#Add dire and radiant heroes to DataFrame under their own columns
for i in range(len(np.array(radiant_hero_column)[0])):
    dota2_test_df['radiant_heroes'+str(i)] = np.array(radiant_hero_column)[:,i]
    dota2_test_df['dire_heroes'+str(i)] = np.array(dire_hero_column)[:,i]

In [28]:
#save data to csv file so that we have it on disk
dota2_test_df.to_csv(os.path.join('dota2_test_data',
                                        str(dota2_test_df['match_id'][0])+
                                        '-'+
                                        str(dota2_test_df['match_id'][499])+'.csv'))

# Load test data that is newest for match_id checking

In [127]:
#Run this if the test data is needed to allow for proper test_data collection
#load newest file so that last_match_id is accurate? Unclear if this is the best strategy... 
#maybe should look at loading all then looking at match_id
dota2_data_files = glob.glob(os.path.join('dota2_test_data','*.csv'))
dota2_data_files.sort(key=os.path.getmtime)

dota2_test_df_from_file = pd.read_csv(dota2_data_files[0], usecols=range(1,13))
dota2_test_df_from_file.style

#purge 0 values from hero_id
dota2_test_df_from_file = dota2_test_df_from_file[dota2_test_df_from_file.radiant_heroes0 != 0]

#make old_match_list for easy checking of match_id
old_match_list = list(dota2_test_df_from_file['match_id'])

# Get 500 matches of each hero id (and trim duplicates)

In [64]:

hero_list = interface('IEconDOTA2_'+dota2_id).GetHeroes(itemizedonly = True)
#print(hero_list)

## Get match_id of all the matches

In [96]:
#get matches data from dota2 API and put IDs into a dataframe
num_matches_to_request = 500

match_list = []
#Loop through all hero_id values
#There are 116 heroes, but ids 105, 116-118 are not occupied
for i in range(1,hero_list['result']['count']+1):
    
    #better option is to build a vocabulary
    if i == 105:
        j = 121
    elif i == 115:
        j = 120
    elif i == 116:
        j = 119
    else:
        j = i
    
    #Get first match for each hero out of loop so that we can pull more than 100 games per hero
    #by utilizing start_at_match_id option
    matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(hero_id = j,
                                                                   game_mode = 1,
                                                                   skill_level = 3,
                                                                   min_players = 10,
                                                                   matches_requested = 100)
    
    #Append matches to matchlist
    #if lobby_type is public/ranked matching (0 or 7)
    for match in matches['result']['matches']:
        if match['lobby_type']==0 or match['lobby_type']==7:
            match_list.append(match['match_id'])
            
    current_num = 1
    
    if len(match_list)==0:
        print('No matches returned\nWait a bit and retry\n\n    GabeN Bless')
        break
    
    while current_num <= num_matches_to_request:
        #sleep so we don't overload the servers
        time.sleep(1)

        current_num += 100
        matches = interface('IDOTA2Match_' + dota2_id).GetMatchHistory(hero_id = j,
                                                                       start_at_match_id = match_list[-1]-1,
                                                                       game_mode = 1,
                                                                       skill_level = 2,
                                                                       min_players = 10,
                                                                       matches_requested = 100)
        
        #Append matches to matchlist
        #if lobby_type is public/ranked matching (0 or 7)
        for match in matches['result']['matches']:
            if match['lobby_type']==0 or match['lobby_type']==7:
                match_list.append(match['match_id'])


## Purge duplicates and add to dataframe

In [97]:
#Make dataframe of match_id values using dict.fromkeys(list) to remove duplicates
dota2_training_df = pd.DataFrame()
dota2_training_df['match_id'] = list(dict.fromkeys(match_list))

dota2_training_df.describe()

,match_id
count,1.602500e+04
mean,4.364969e+09
std,2.075179e+04
min,4.364875e+09
25%,4.364957e+09
50%,4.364971e+09
75%,4.364984e+09
max,4.365037e+09


## Get match details for matches

In [98]:
#initialize data columns if you're starting a match_detail getting batch
#if trying to finish one that broke unexpectedly then don't!
radiant_win_column = []
radiant_hero_column = []
dire_hero_column = []
match_details = []

In [99]:
#get match details from match_ids obtained
#add radiant_win column
#add heroes split between dire and radiant

for i,match_id in enumerate(dota2_training_df[:]['match_id']):
    time.sleep(1)
    match_details.append(interface('IDOTA2Match_' + dota2_id).GetMatchDetails(match_id = str(match_id)))
    
    try:
        radiant_win_column.append(match_details[i]['result']['radiant_win'])
    except:
        print('server error at match_id: ' + str(match_id))
        print('element:' + str(i))
        break
    
    radiant_hero_row = []
    dire_hero_row = []
    for player in match_details[i]['result']['players']:
        #First bit of 8-bit integer player_slot is 1 if the player was on dire
        #so if player was on dire then player_slot > 128
        if player['player_slot']/128<1:
            radiant_hero_row.append(player['hero_id'])
        else:
            dire_hero_row.append(player['hero_id'])
    
    radiant_hero_column.append(radiant_hero_row)
    dire_hero_column.append(dire_hero_row)

KeyboardInterrupt: 

In [ ]:
print(len(radiant_hero_column))
print(len(dota2_training_df['match_id']))

### Run if the match_detail fetching broke and the df length is different from the data column lengths

In [92]:
match_id_list = []
match_details = list(match_details)
radiant_win_column = []
radiant_hero_column = []
dire_hero_column = []

for i,match in enumerate(match_details):
    
    match_id_list.append(match['result']['match_id'])
    radiant_win_column.append(match['result']['radiant_win'])
    
    radiant_hero_row = []
    dire_hero_row = []
    for player in match['result']['players']:
        #First bit of 8-bit integer player_slot is 1 if the player was on dire
        #so if player was on dire then player_slot > 128
        if player['player_slot']/128<1:
            radiant_hero_row.append(player['hero_id'])
        else:
            dire_hero_row.append(player['hero_id'])
            
    radiant_hero_column.append(radiant_hero_row)
    dire_hero_column.append(dire_hero_row)
dota2_training_df = pd.DataFrame()
dota2_training_df['match_id'] = match_id_list

## Process and save data

In [93]:
#Add radiant_win column to Dataframe
dota2_training_df['radiant_win'] = 1*np.array(radiant_win_column)

#Add dire and radiant heroes to DataFrame under their own columns
for i in range(len(np.array(radiant_hero_column)[1])):
    print(i)
    dota2_training_df['radiant_heroes'+str(i)] = np.array(radiant_hero_column)[:,i]
    dota2_training_df['dire_heroes'+str(i)] = np.array(dire_hero_column)[:,i]

0
1
2
3
4


In [94]:
#purge 0 values from hero_id
dota2_training_df = dota2_training_df[dota2_training_df.radiant_heroes0 != 0]


In [95]:
#save data to csv file so we have it on disk
dota2_training_df.to_csv(os.path.join('dota2_training_data',
                                        'skill_level=3',
                                        str(dota2_training_df['match_id'].min())+
                                        '-'+
                                        str(dota2_training_df['match_id'].max())+
                                        '.csv'))

## Export data as json - not necessary since all data is post-match
Maybe useful if I want to change the target from win to gold/xp difference on each team?

In [63]:
outfile = os.path.join('dota2_training_data',
                                        'match_id'+
                                        str(dota2_training_df['match_id'][0])+
                                        '-'+
                                        str(list(dota2_training_df['match_id'])[-1])+
                                        '.json')

match_details_dict_list = [dict(match_detail) for match_detail in match_details]
import json
with open(outfile, 'w') as fout:
    json.dump(match_details_dict_list, fout)

# Build TFRecords file
Puts all training data in the training_data folder into one TFRecords file
Do this to nicely handle the features that are numpy arrays (radiant_heroes and dire_heroes)

In [5]:
dota2_df = None
for filename in glob.glob(os.path.join('dota2_training_data','*','*.csv')):
    if type(dota2_df)==None:
        dota2_df = pd.read_csv(filename, usecols=range(1,13))
    else:
        dota2_df = pd.concat([dota2_df, pd.read_csv(filename, usecols=range(1,13))],ignore_index=True)

dota2_df = dota2_df.reindex(np.random.permutation(dota2_df.index))
dota2_df.describe()

,match_id,radiant_win,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,7.159920e+05,715992.000000,715992.000000,715992.000000,715992.000000,715992.000000,715992.000000,715992.000000,715992.000000,715992.000000,715992.000000,715992.000000
mean,4.303050e+09,0.544163,51.439217,51.425590,51.478145,51.529193,51.457513,51.546316,51.491648,51.517291,51.315900,51.401285
std,3.214784e+07,0.498046,34.947077,34.877813,34.943115,34.949683,34.980628,34.961331,34.945455,34.945621,34.927541,34.851845
min,4.252092e+09,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,4.276715e+09,0.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,4.295993e+09,1.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
75%,4.326205e+09,1.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
max,4.364279e+09,1.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000


In [6]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [7]:
def convert_to_tfrecords(filename, examples, targets):
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(filename)
    
    radiant_hero_array = np.array(examples['radiant_heroes'])
    dire_hero_array = np.array(examples['dire_heroes'])
    target_array = np.array(targets['radiant_win'])
    
    for i in range(len(radiant_hero_array[:])):
        # print how many games are saved every 5000 games
        if not i % 5000:
            print('Train data: %d/%d' % (i, len(examples)))
            sys.stdout.flush()
            
        # Load the image
        radiant_heroes = radiant_hero_array[:][i].tostring()
        dire_heroes = dire_hero_array[:][i].tostring()
        target = target_array[i]
        
        # Create a feature
        feature = {'radiant_heroes': _bytes_feature(tf.compat.as_bytes(radiant_heroes)),
                   'dire_heroes': _bytes_feature(tf.compat.as_bytes(dire_heroes)),
                   'targets': _int64_feature(target)}
        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))
    
        # Serialize to string and write on the file
        writer.write(example.SerializeToString())
    
    writer.close()
    sys.stdout.flush()

In [8]:
def preprocess_features(dota2_df):
    """Take dota2_df and create a dataframe containing only the features for our model
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                processed_df: pandas DataFrame containing only feature columns
    """
    
    #Use arrays of heroes for each team as features rather than each hero
    #individually to help model fit?
    #This is an attempt to make the model more similar to the movie review
    #text analysis example in the Google ML Crash Course
    processed_df = pd.DataFrame()
    
    processed_df['radiant_heroes'] = list(np.array(dota2_df.loc[:,['radiant_heroes0',
                                                'radiant_heroes1','radiant_heroes2',
                                                'radiant_heroes3','radiant_heroes4']]))
    processed_df['dire_heroes'] = list(np.array(dota2_df.loc[:,['dire_heroes0',
                                    'dire_heroes1','dire_heroes2',
                                    'dire_heroes3','dire_heroes4']]))
    
    ##only features (to start with) are the heroes in the game
    #processed_df = dota2_df[['radiant_heroes0', 'radiant_heroes1',
    #                   'radiant_heroes2', 'radiant_heroes3', 'radiant_heroes4',
    #                    'dire_heroes0', 'dire_heroes1', 'dire_heroes2',
    #                    'dire_heroes3', 'dire_heroes4']]
    
    ##create two synthetic features that is the product of all heroes for each team
    #processed_df['radiant_hero_product'] = dota2_df['radiant_heroes0']*dota2_df['radiant_heroes1']*dota2_df['radiant_heroes2']dota2_df['radiant_heroes3']*dota2_df['radiant_heroes4']
    #processed_df['dire_hero_product'] = dota2_df['dire_heroes0']*dota2_df['dire_heroes1']*dota2_df['dire_heroes2']dota2_df['dire_heroes3']*dota2_df['dire_heroes4']
    
    return processed_df
    
def preprocess_targets(dota2_df):
    """Take dota2_df and create a dataframe containing only the targets for our model
            Args: 
                dota2_df: Dataframe containing dota2 training and test data
            returns: 
                target_df: pandas DataFrame containing only the target column
    """
    target_df = pd.DataFrame()
    target_df['radiant_win'] = dota2_df['radiant_win']
    
    return target_df

### Visualize data for each set

In [17]:
plotting_df = dota2_df

display.display(plotting_df.iloc[:train_len,:].describe())
display.display(plotting_df.iloc[train_len:train_len+test_len,:].describe())
display.display(plotting_df.iloc[train_len+test_len:,:].describe())


,match_id,radiant_win,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,3.057300e+04,30573.000000,30573.000000,30573.000000,30573.000000,30573.000000,30573.000000,30573.000000,30573.000000,30573.000000,30573.000000,30573.000000
mean,4.259099e+09,0.553397,51.428025,51.400321,51.577339,51.639584,51.170248,51.750008,51.859157,51.437281,50.886043,51.326890
std,2.941378e+06,0.497149,34.716924,34.804333,34.715398,34.696065,34.801647,34.838768,34.848814,34.682248,34.829748,34.746297
min,4.252092e+09,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.258275e+09,0.000000,21.000000,21.000000,21.000000,22.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,4.259846e+09,1.000000,44.000000,44.000000,44.000000,44.000000,44.000000,45.000000,45.000000,44.000000,44.000000,44.000000
75%,4.262045e+09,1.000000,83.000000,84.000000,83.000000,83.000000,83.000000,84.000000,84.000000,83.000000,83.000000,83.000000
max,4.262137e+09,1.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000


,match_id,radiant_win,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,2.038200e+04,20382.000000,20382.000000,20382.000000,20382.000000,20382.000000,20382.000000,20382.000000,20382.000000,20382.000000,20382.000000,20382.000000
mean,4.259062e+09,0.555000,50.903886,51.340055,51.604896,51.229173,51.229418,51.677510,51.351781,51.666814,50.969826,51.530174
std,2.952171e+06,0.496978,35.026320,34.583701,34.874450,34.755532,34.714573,34.658889,34.720341,34.955004,34.861148,34.659054
min,4.252092e+09,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.258274e+09,0.000000,20.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
50%,4.259844e+09,1.000000,44.000000,44.000000,44.000000,44.000000,44.000000,45.000000,44.000000,44.000000,44.000000,45.000000
75%,4.262043e+09,1.000000,83.000000,82.000000,84.000000,83.000000,83.000000,84.000000,83.000000,84.000000,83.000000,83.000000
max,4.262137e+09,1.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000


,match_id,radiant_win,radiant_heroes0,dire_heroes0,radiant_heroes1,dire_heroes1,radiant_heroes2,dire_heroes2,radiant_heroes3,dire_heroes3,radiant_heroes4,dire_heroes4
count,1.019100e+04,10191.000000,10191.000000,10191.000000,10191.000000,10191.000000,10191.000000,10191.000000,10191.000000,10191.000000,10191.000000,10191.000000
mean,4.259046e+09,0.551859,50.979492,51.188598,51.580316,51.520263,51.165146,51.634481,50.633696,51.244235,51.554313,51.289275
std,3.005546e+06,0.497328,34.675734,34.753109,34.672125,34.794969,34.593417,34.769220,34.486406,34.796130,34.606377,34.520960
min,4.252092e+09,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,4.258274e+09,0.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,22.000000,21.000000
50%,4.259845e+09,1.000000,44.000000,44.000000,45.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
75%,4.262045e+09,1.000000,82.000000,82.000000,83.000000,84.000000,82.000000,84.000000,81.000000,84.000000,83.000000,83.000000
max,4.262128e+09,1.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000


## Make tfrecords files

In [9]:
training_frac = 0.7
train_len = int(len(dota2_df) * training_frac)
test_len = int((len(dota2_df) - train_len)/1.5)
validation_len = len(dota2_df) - train_len - test_len

train_features = preprocess_features(dota2_df.iloc[:train_len,:])
train_targets = preprocess_targets(dota2_df.iloc[:train_len,:])

test_features = preprocess_features(dota2_df.iloc[train_len:train_len+test_len,:])
test_targets = preprocess_targets(dota2_df.iloc[train_len:train_len+test_len,:])

validation_features = preprocess_features(dota2_df.iloc[train_len+test_len:,:])
validation_targets = preprocess_targets(dota2_df.iloc[train_len+test_len:,:])

convert_to_tfrecords(os.path.join('dota2_training_data', 'dota2_training_data.tfrecords'),
                     train_features,
                     train_targets)

convert_to_tfrecords(os.path.join('dota2_test_data', 'dota2_test_data.tfrecords'),
                     test_features,
                     test_targets)

convert_to_tfrecords(os.path.join('dota2_validation_data', 'dota2_validation_data.tfrecords'),
                     validation_features,
                     validation_targets)


Train data: 0/501194
Train data: 5000/501194
Train data: 10000/501194
Train data: 15000/501194
Train data: 20000/501194
Train data: 25000/501194
Train data: 30000/501194
Train data: 35000/501194
Train data: 40000/501194
Train data: 45000/501194
Train data: 50000/501194
Train data: 55000/501194
Train data: 60000/501194
Train data: 65000/501194
Train data: 70000/501194
Train data: 75000/501194
Train data: 80000/501194
Train data: 85000/501194
Train data: 90000/501194
Train data: 95000/501194
Train data: 100000/501194
Train data: 105000/501194
Train data: 110000/501194
Train data: 115000/501194
Train data: 120000/501194
Train data: 125000/501194
Train data: 130000/501194
Train data: 135000/501194
Train data: 140000/501194
Train data: 145000/501194
Train data: 150000/501194
Train data: 155000/501194
Train data: 160000/501194
Train data: 165000/501194
Train data: 170000/501194
Train data: 175000/501194
Train data: 180000/501194
Train data: 185000/501194
Train data: 190000/501194
Train data: